In [1]:
import zipfile
import pandas as pd
import os
import torch
from tqdm.notebook import tqdm
import csv
import requests
from sklearn.model_selection import train_test_split
import zipfile
from torch.utils.data import Dataset
from PIL import Image
from transformers import TrOCRProcessor, ViTImageProcessor, BertTokenizer, VisionEncoderDecoderModel
from datasets import load_dataset
from transformers import TrOCRProcessor
from transformers import VisionEncoderDecoderModel
from torch.utils.data import DataLoader
from datasets import load_metric


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
current_directory = os.getcwd()

In [3]:
def download_file(url, save_path):
    print(f"Downloading File at {save_path}")
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for non-2xx status codes
        with open(save_path, "wb") as file:
            file.write(response.content)
        print("Download completed!")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while downloading the file: {e}")



In [4]:
# Download IAM DATA SET
# file_url = "https://storage.googleapis.com/ayuda/IAM-data.zip"
# save_location =  os.path.join(current_directory, 'IAM-data.zip')
# download_file(file_url, save_location)

# img_zip_path = os.path.join(current_directory, 'IAM-data.zip')
# zip_ref = zipfile.ZipFile(img_zip_path, 'r')
# zip_ref.extractall(current_directory)
# zip_ref.close()

In [5]:
def create_new_text_directory():
    file_path = "IAM-data/iam_lines_gt.txt"
    file_path = os.path.join(current_directory, file_path)

    with open(file_path, 'r') as file:
        # Read the entire file
        content = file.read()
        content = content.replace("\n", "\t")
        content = content.replace("\t\t", "\n")

    file_path = "IAM-data/iam_lines_ours.txt"
    file_path = os.path.join(current_directory, file_path)
    with open(file_path, 'w') as file:
        file.write(content)
    return file_path
file_path = create_new_text_directory()

In [6]:
df = pd.read_csv(file_path, delimiter='\t', header=None, names=["file_name", "text"], encoding='utf-8', quoting=csv.QUOTE_NONE)
df['length'] = df['text'].str.len()

df.shape

(7458, 3)

In [7]:
train_df, test_df = train_test_split(df, test_size=0.2,random_state=42)
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
test_df, eval_df = train_test_split(test_df, test_size=0.5,random_state=42)
test_df.reset_index(drop=True, inplace=True)
eval_df.reset_index(drop=True, inplace=True)

In [8]:
print(train_df.shape)
print(test_df.shape)
print(eval_df.shape)

(5966, 3)
(746, 3)
(746, 3)


In [9]:
class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels), "file_name": file_name}
        return encoding

In [10]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [13]:
train_dataset = IAMDataset(root_dir=os.path.join(current_directory, 'IAM-data/img/'),
                           df=train_df,
                           processor=processor)
test_dataset = IAMDataset(root_dir=os.path.join(current_directory, 'IAM-data/img/'),
                           df=test_df,
                           processor=processor)
eval_dataset = IAMDataset(root_dir=os.path.join(current_directory, 'IAM-data/img/'),
                           df=eval_df,
                           processor=processor)

In [14]:
print("Number of test examples:", len(test_dataset))

Number of test examples: 746


In [15]:
test_dataloader = DataLoader(test_dataset, batch_size=8)

In [16]:
# Download MODEL WEIGHTS
# file_url = "https://storage.googleapis.com/ayuda/checkpoint-9000.zip"
# save_location =  os.path.join(current_directory, 'checkpoint-9000.zip')
# download_file(file_url, save_location)

# model_zip = "checkpoint-9000.zip"
# img_zip_path = os.path.join(current_directory, model_zip)
# zip_ref = zipfile.ZipFile(img_zip_path, 'r')
# zip_ref.extractall(current_directory)
# zip_ref.close()

In [24]:
model = VisionEncoderDecoderModel.from_pretrained("checkpoint-10000")

OSError: checkpoint-10000 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [19]:
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fea

In [20]:
cer = load_metric("cer")

C:\Users\jl\AppData\Local\Temp\ipykernel_19752\2134553922.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer = load_metric("cer")


In [21]:
print("Running evaluation...")

results = []
for batch in tqdm(test_dataloader):
    # predict using generate
    pixel_values = batch["pixel_values"].to(device)
    outputs = model.generate(pixel_values)

    # decode
    pred_str = processor.batch_decode(outputs, skip_special_tokens=True)
    labels = batch["labels"]
    labels[labels == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels, skip_special_tokens=True)
    
    # add batch to metric
    cer.add_batch(predictions=pred_str, references=label_str)
    
    results = results + [(file_name, label, pred) for file_name, pred, label in zip(batch["file_name"], pred_str, label_str) ]

final_score = cer.compute()

Running evaluation...


  0%|          | 0/94 [00:00<?, ?it/s]

C:\Users\jl\Miniconda3\envs\torch\lib\site-packages\transformers\generation\utils.py:1346: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
print("Character error rate on test set:", final_score)

In [23]:
print("Character error rate on test set:", final_score)

Character error rate on test set: 0.09767412302999491


In [ ]:
for img, label, pred in results:
    # Write content to the file
    with open(os.path.join(current_directory, file_path), 'w') as file:
        file.write(f"{img}\t||\t{label}\t||\t{pred}\n")
        